In [ ]:
import json
from urllib.request import urlopen

import pandas as pd
import plotly.express as px

In [ ]:
excel_url = "https://github.com/MAROON-LABKOM/script-praktikum/blob/master/pemrograman-data-analisis/kawalcovid19-small-pda.xlsx?raw=true"
geojson_url = "https://github.com/MAROON-LABKOM/script-praktikum/blob/master/pemrograman-data-analisis/indonesia-province-simple.json?raw=true"

daily_cases_df = pd.read_excel(excel_url, sheet_name="daily_cases", parse_dates=["timestamp"])
province_df = pd.read_excel(excel_url, sheet_name="province_active_case_unpivot", parse_dates=["timestamp"])

with urlopen(geojson_url) as response:
    indonesia_provinces = json.load(response)

In [ ]:
province_df["province"] = province_df["province"].str.upper()
daily_cases_df["total_vaccination"] = daily_cases_df.iloc[:, 8:].sum(axis=1)

In [ ]:
indonesia_provinces["features"][0]["properties"]

{'ID': 1,
 'Propinsi': 'IRIAN JAYA TIMUR',
 'SUMBER': 'Peta Dasar BAKOSURTANAL Skala 1 : 250.000',
 'kode': 85}

In [ ]:
daily_cases_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 657 entries, 0 to 656
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   timestamp          657 non-null    datetime64[ns]
 1   daily_case         657 non-null    int64         
 2   active_case        657 non-null    int64         
 3   cured              657 non-null    int64         
 4   deceased           657 non-null    int64         
 5   test_cov           657 non-null    int64         
 6   test_negative      657 non-null    int64         
 7   test_positive      186 non-null    float64       
 8   vac_first_dose     341 non-null    float64       
 9   vac_second_dose    327 non-null    float64       
 10  vac_third_dose     124 non-null    float64       
 11  total_vaccination  657 non-null    float64       
dtypes: datetime64[ns](1), float64(5), int64(6)
memory usage: 61.7 KB


In [ ]:
daily_cases_df.head()

,timestamp,daily_case,active_case,cured,deceased,test_cov,test_negative,test_positive,vac_first_dose,vac_second_dose,vac_third_dose,total_vaccination
0,2020-03-02,2,2,0,0,339,335,2.0,NaN,NaN,NaN,0.0
1,2020-03-03,0,2,0,0,2,2,0.0,NaN,NaN,NaN,0.0
2,2020-03-04,0,2,0,0,370,354,2.0,NaN,NaN,NaN,0.0
3,2020-03-05,0,2,0,0,18,17,0.0,NaN,NaN,NaN,0.0
4,2020-03-06,2,4,0,0,432,405,4.0,NaN,NaN,NaN,0.0


In [ ]:
province_df.head()

,timestamp,province,active_case
0,2020-03-18,ACEH,0
1,2020-03-18,BALI,1
2,2020-03-18,BANTEN,17
3,2020-03-18,BANGKA BELITUNG,0
4,2020-03-18,BENGKULU,0


In [ ]:
start_date = pd.to_datetime("2021-09-01")
end_date = pd.to_datetime("2021-09-30")

daily_cases_subset_df = daily_cases_df[(daily_cases_df["timestamp"] >= start_date) & (daily_cases_df["timestamp"] <= end_date)]
province_subset_df = province_df[(province_df["timestamp"] >= start_date) & (province_df["timestamp"] <= end_date)]

In [ ]:
total_kasus = daily_cases_subset_df["daily_case"].sum()
meninggal_dunia = daily_cases_subset_df["deceased"].sum()
total_dosis_vaksin = daily_cases_subset_df["total_vaccination"].sum()

print("Total Kasus:", total_kasus)
print("Meninggal Dunia:", meninggal_dunia)
print("Total Dosis Vaksin:", total_dosis_vaksin)

Total Kasus: 125303
Meninggal Dunia: 8916
Total Dosis Vaksin: 1829932466.3333333


In [ ]:
fig = px.line(daily_cases_subset_df, x="timestamp", y="daily_case", title="Kasus Harian", line_shape='spline')
fig.show()

In [ ]:
fig = px.line(daily_cases_subset_df, x="timestamp", y="deceased", title="Meninggal Dunia", line_shape='spline')
fig.show()

In [ ]:
fig = px.line(daily_cases_subset_df, x="timestamp", y="total_vaccination", title="Total Dosis Vaksin", line_shape='spline')
fig.show()

In [ ]:
province_subset_totals_df = province_subset_df.groupby("province").sum().reset_index()
province_subset_totals_df

In [ ]:
fig = px.choropleth(province_subset_totals_df, geojson=indonesia_provinces, color="active_case", locations="province", featureidkey="properties.Propinsi")
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()